In [1]:
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize
import functools
import json
import ast
import re

from sklearn.preprocessing import MultiLabelBinarizer
from autoimpute.imputations import MultipleImputer
from dython.nominal import cramers_v, theils_u 

import sys

In [2]:
# Packages

# %pip install autoimpute
# %pip install pixiedust
# %pip install impyute
# %pip install datawig
# %pip install dython

In [3]:
# Dataset paths
business_path = 'data/yelp_academic_dataset_business.json'
checkin_path = 'data/yelp_academic_dataset_checkin.json'
review_path = 'data/yelp_academic_dataset_review.json'
tip_path = 'data/yelp_academic_dataset_tip.json'
user_path = 'data/yelp_academic_dataset_user.json'

# Business data

За да добиеме слика околу тоа како изгледа податочното множество подолу се дадени атрибутите и нивното значење.

**business_id** - уникатен идентификатор за бизнисот составен од 22 карактери <br>
**name** - име на бизнисот <br>
**address** - адреса на бизнисот <br>
**city** - град <br>
**state** - код на state-от, 2 карактери <br>
**postal_code** - поштенски број <br>
**latitude**, **longitude** - координати на бизнисот <br>
**stars** - оценка заокружена на половина ѕвезда <br>
**review_count** - број на review-а <br>
**is_open** - дали бизнисот е отворен или затворен. Вредностите се 0 и 1 за отворен или затворен соодветно <br>
**attributes** - атрибути за биснисот. Вредноста е објект составен од име на атрибутот и вредност која може да биде проста или објект <br>
**categories** - листа на категории <br>
**hours** - објект кој мапира ден во неделата со работно време во 24 часовен формат <br>

In [4]:
business = pd.read_json(business_path, lines=True)

In [5]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


Пред да поминеме на чистење на потоците би требале да се справиме со атритубите `attributes` и `hour` кои претставуваат објекти, но и со `categories` што содржи листа од вредности. Соодветно, за полесно справување со овие атрибути истите ќе ги претавиме како посебни колони зависно од нивната вредсност.

Бидејќи ваквата операција ќе генерира голем број на вредности коишто недостасуваат, добро е да имаме увид во бројот на вредности кои недостасуваат пред "нормализација" на json вредностите:

In [6]:
print(business.isna().sum().to_string())

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      29045
categories        524
hours           44843


**attributes** - атрибути за биснисот. Вредноста е објект составен од име на атрибутот и вредност која може да биде проста или објект <br>

In [7]:
def is_json(value):
    try:
        json_object = ast.literal_eval(value)
    except ValueError as e:
        return False
    
    if isinstance(json_object, dict):
        return True
    
    return False

def attributes_mapper(value):
    if value is None:
        return {}
    else:
        return {k: ast.literal_eval(v) if is_json(v) else v for k, v in value.items()}
            

In [8]:
def column_unique_values(df, cols):
    filtered_df = df[cols]
    for col in filtered_df:
        print('{:<40} {:>8}'.format(col, str(filled_attr[col].unique())))

In [9]:
attribute_values = business.attributes.apply(attributes_mapper).tolist()
normalized_attributes = json_normalize(attribute_values, sep='_')
normalized_attributes = normalized_attributes.add_prefix('attr_')
# business = pd.concat([business, normalized_attributes], axis=1)
# business = business.drop('attributes', axis=1)
# unique_attributes = functools.reduce(lambda x, y: x.union(list(y.keys())), attribute_values, set())

In [10]:
normalized_attributes.columns.tolist()

['attr_BusinessAcceptsCreditCards',
 'attr_BikeParking',
 'attr_GoodForKids',
 'attr_ByAppointmentOnly',
 'attr_RestaurantsPriceRange2',
 'attr_BusinessParking_garage',
 'attr_BusinessParking_street',
 'attr_BusinessParking_validated',
 'attr_BusinessParking_lot',
 'attr_BusinessParking_valet',
 'attr_DogsAllowed',
 'attr_WiFi',
 'attr_RestaurantsAttire',
 'attr_RestaurantsTakeOut',
 'attr_NoiseLevel',
 'attr_RestaurantsReservations',
 'attr_RestaurantsGoodForGroups',
 'attr_BusinessParking',
 'attr_HasTV',
 'attr_Alcohol',
 'attr_RestaurantsDelivery',
 'attr_OutdoorSeating',
 'attr_Caters',
 'attr_WheelchairAccessible',
 'attr_AcceptsInsurance',
 'attr_RestaurantsTableService',
 'attr_HappyHour',
 'attr_Ambience_touristy',
 'attr_Ambience_hipster',
 'attr_Ambience_romantic',
 'attr_Ambience_intimate',
 'attr_Ambience_trendy',
 'attr_Ambience_upscale',
 'attr_Ambience_classy',
 'attr_Ambience_casual',
 'attr_GoodForMeal_dessert',
 'attr_GoodForMeal_latenight',
 'attr_GoodForMeal_lunch'

In [11]:
object_rgx = re.compile('^(attr_([^_]+))_')
object_attr = list(set([object_rgx.match(attr_name).group(2) for attr_name in normalized_attributes.columns.tolist() 
                    if object_rgx.match(attr_name)]))

attr_descendants = {attr: list(filter(lambda x, y=attr: re.compile(f'^attr_{y}_').match(x), 
                            normalized_attributes.columns.tolist())) for attr in object_attr}

In [12]:
# Check unique values of nested attributes
attr_values = { attr : set() for attr in object_attr }
for attr in business.attributes:
    if attr is None:
        continue
    name_mapper= lambda x: '{obj}' if x.startswith('{') else x
    
    for attr_name in object_attr: 
        if attr_name in attr:
            attr_values[attr_name].add(name_mapper(attr[attr_name]))

In [13]:
object_attr

['BestNights',
 'HairSpecializesIn',
 'Music',
 'DietaryRestrictions',
 'GoodForMeal',
 'BusinessParking',
 'Ambience']

In [14]:
attr_values

{'BestNights': {'None', '{obj}'},
 'HairSpecializesIn': {'None', '{obj}'},
 'Music': {'None', '{obj}'},
 'DietaryRestrictions': {'None', '{obj}'},
 'GoodForMeal': {'None', '{obj}'},
 'BusinessParking': {'None', '{obj}'},
 'Ambience': {'None', '{obj}'}}

In [15]:
normalized_attributes.loc[normalized_attributes['attr_BusinessParking'] == 'None', attr_descendants['BusinessParking']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_BestNights'] == 'None', attr_descendants['BestNights']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_DietaryRestrictions'] == 'None', attr_descendants['DietaryRestrictions']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_Ambience'] == 'None', attr_descendants['Ambience']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_GoodForMeal'] == 'None', attr_descendants['GoodForMeal']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_Music'] == 'None', attr_descendants['Music']] = 'None'
normalized_attributes.loc[normalized_attributes['attr_HairSpecializesIn'] == 'None', attr_descendants['HairSpecializesIn']] = 'None'
normalized_attributes = normalized_attributes.drop([f'attr_{attr}' for attr in object_attr], axis=1)

**hours** - објект кој мапира ден во неделата со работно време во 24 часовен формат <br>

Тука треба да направиме разлика помеѓу оние коишто немаат поставено работно време и оние коишто не работат во одредени денови. Соодветно, бизнисите коишто не работат во одредени денови за тој ден ќе имаат вредност *`'Closed'`*.

In [16]:
functools.reduce(lambda x, y: x.union(list(y.keys())) if y is not None else x.add('None') or x, 
                 business.hours.tolist(), set())

{'Friday',
 'Monday',
 'None',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday'}

In [17]:
def hours_mapper(hours):
    if hours is None:
        return {}
    
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return {day: 'Closed' if day not in hours.keys() else hours[day] for day in days}

In [18]:
hours_values = business.hours.apply(hours_mapper).tolist()
normalized_hours = json_normalize(hours_values, sep='_')

In [19]:
normalized_hours

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,10:0-18:0,11:0-20:0,10:0-18:0,11:0-20:0,11:0-20:0,11:0-20:0,13:0-18:0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7:0-16:0,7:0-16:0,7:0-16:0,7:0-16:0,7:0-16:0,Closed,Closed
4,0:0-0:0,9:0-16:0,9:0-16:0,9:0-16:0,9:0-16:0,Closed,Closed
...,...,...,...,...,...,...,...
209388,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,Closed
209389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209390,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-19:0
209391,0:0-0:0,7:0-19:0,7:0-19:0,7:0-19:0,7:0-19:0,9:0-19:0,9:0-19:0


**categories** - листа на категории <br>

Оваа колона ќе ја поделам на онолку колку колку што категории постојат. Вредноста за соодветната инстанца ќе биде `1` или `0` во зависност од тоа дали бизнисот припаѓа или не на категоријата соодветно. Со инстанците коишто немаат асоцирана вредност ќе се справиме подоцна. 

In [20]:
def categories_mapper(categories):
    if categories is None:
        return []
    
    name_filter = lambda x: x if x.isalpha() else False
    categories_list = [''.join(filter(name_filter, category)) for category in categories.split(', ')]
    
    return categories_list

In [21]:
categories_values = business.categories.apply(categories_mapper)
business_categories = functools.reduce(lambda x, y: x.union(categories_mapper(y)) or x, 
                                       business.categories, set())

In [22]:
mlb = MultiLabelBinarizer(sparse_output=True)
categories_matrix = mlb.fit_transform(categories_values)
categories_matrix = categories_matrix.astype('float64')
categories_matrix[business[business['categories'].isnull()].index, :] = np.nan
normalized_categories = pd.DataFrame.sparse.from_spmatrix(categories_matrix, columns=mlb.classes_)
normalized_categories = normalized_categories.add_prefix('ctg_')

/Users/darko/anaconda3/envs/nlp_finki/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [23]:
print('Number of categories: ', len(business_categories))

Number of categories:  1336


In [24]:
business_norm = pd.concat([business, normalized_attributes, normalized_categories, normalized_hours], axis=1)
business_norm = business_norm.drop(['attributes', 'categories', 'hours'], axis=1)

Моментална состојба на вредностите коишто недостасуваат.

In [25]:
print(business_norm.isna().sum().to_string())

business_id                                    0.0
name                                           0.0
address                                        0.0
city                                           0.0
state                                          0.0
postal_code                                    0.0
latitude                                       0.0
longitude                                      0.0
stars                                          0.0
review_count                                   0.0
is_open                                        0.0
attr_BusinessAcceptsCreditCards            87156.0
attr_BikeParking                          119628.0
attr_GoodForKids                          140858.0
attr_ByAppointmentOnly                    148594.0
attr_RestaurantsPriceRange2                98105.0
attr_BusinessParking_garage                94816.0
attr_BusinessParking_street                95005.0
attr_BusinessParking_validated             94816.0
attr_BusinessParking_lot       

Сега имаме множество коешто е во форма на матрица и е поразбирливо за моделите од машинско учење. Сепак, за да ги подобриме нашите резултати треба најпрвин да се справиме со вредностите коишто недостасуваат, а потоа и да ги конвертираме податоците за нивно полесна обработка.

## Чистење на податоци

***

Од првичната состојба може да се согледа дека имаме вредности коишто недостасуваат во колоните `attributes`, `categories` и `hours`. По нормализација на форматот во којшто се сместени податоците, од овие три колони произлегуваат дополнителни вредности коишто недостигаат, па истите мораме да ги пополниме. Во продолжение, за секоја колона којашто произлегува од иницијалните колони ќе се обидеме да ги пополниме `NaN` вредностите, пред се следејќи ги ограничувањата коишто ги наметнува документацијата, но и според останатите податоци во податочното множество.

**attributes** - атрибути за биснисот. Вредноста е објект составен од име на атрибутот и вредност која може да биде проста или објект <br>
**categories** - листа на категории <br>
**hours** - објект кој мапира ден во неделата со работно време во 24 часовен формат <br>


Најпрвин ќе започнеме со `attributes` вредностите коишто недостасуваат како резултат од нормализацијата на форматот. Различните атрибути коишто ги имаат бизнисите се пополнети со вредностите коишто се специфицирани во клуч-вредност паровите, но како резултат на непополнување на сите атрибути имаме `NaN` вредности. Ваквите атрибути најверојатно претставуваат дека бизнисите не го нудат/имаат она што го опишува истиот, но бидејќи значењето не е добро документирано истото ќе го поставам на `Unknown`.

In [26]:
business_norm.loc[~business['attributes'].isna(), normalized_attributes.columns.tolist()] = business_norm.loc[~business['attributes'].isna(), normalized_attributes.columns.tolist()].fillna('Unknown')

За одредување на категоријата на бизнисот, покрај останатите карактеристики, добро е да се земат во предвид атрибутите на истиот. Токму поради оваа причина, добро е да се има увид во тоа колку инстанци на коишто им недостасува категоријата имаат соодветни атрибути.

In [27]:
business.loc[business.categories.isna()].shape

(524, 14)

In [28]:
business.loc[(business.categories.isna()) & (business.attributes.isna())].shape

(523, 14)

In [29]:
business.loc[(business.categories.isna()) & (business.hours.isna())].shape

(520, 14)

In [30]:
business.loc[(business.categories.isna()) & (business.attributes.isna()) & (business.hours.isna()) ].shape

(519, 14)

Од `524` бизниси без категорија на `523` им недостасуваат атрибутите, а на `520` им недостасува работното време. Тука имаме потенцијален проблем бидејќи овие променливи би можеле да носат силен сигнал во она што би сакале да го предвидиме, односно моделираме. Токму поради ова, потенцијално решение би било да се **отстранат инстанците на коишто им недостасуваат сите три атрибути**, но добра идеја би било да се проба и **пристапот на пополнување со најверојатни вредности**. Ова размислување се должи на потенцијална информација којашто можат да ја носат останатите атрибути.

### Претворање на атрибутите во нумерички

Пред пополнување на вредностите кои недостасуваат, со цел полесно справување со истите, атрибутите ќе ги трансформираме во нумерички. Покрај полесно справување со вредностите коишто недостасуваат, претворањето на атрибутите во нумерички ќе ни овозможи полесно анализирање, но и примена на алгоритми кои работат само со ваквите атрибути. Како и да е, со цел задржување на што е можно повеќе информации потребно е да обрнеме внимание на тоа дали атрибутите се ординални или пак номинални. Вредности на ординалните атрибути ќе ги претворам во подредени броеви.

In [31]:
tCategorical_features = ['city', 'state'] + [col for col in normalized_attributes.columns.tolist() if col in business_norm.columns.tolist()]

In [32]:
filled_attr = business_norm[tCategorical_features]
for col in filled_attr:
    print('{:<40} {:>8} {:1}'.format(col, str(filled_attr[col].unique()), len(filled_attr[col].unique())))

city                                     ['Cornelius' 'Scottsdale' 'Montreal' ... 'ARSENAL' 'Chander' 'Tempe '] 1251
state                                    ['NC' 'AZ' 'QC' 'NV' 'IL' 'ON' 'AB' 'PA' 'WI' 'SC' 'OH' 'CA' 'TX' 'NY'
 'CO' 'XWY' 'GA' 'BC' 'YT' 'HPL' 'AL' 'UT' 'VT' 'WA' 'NE' 'DOW' 'MI' 'FL'
 'AR' 'HI' 'MB' 'OR' 'AK' 'VA' 'CT' 'MO' 'DUR'] 37
attr_BusinessAcceptsCreditCards          ['True' 'Unknown' nan 'False' 'None'] 5
attr_BikeParking                         ['True' 'Unknown' nan 'False' 'None'] 5
attr_GoodForKids                         ['False' 'True' nan 'Unknown' 'None'] 5
attr_ByAppointmentOnly                   ['False' 'True' nan 'Unknown' 'None'] 5
attr_RestaurantsPriceRange2              ['3' 'Unknown' nan '4' '1' '2' 'None'] 7
attr_BusinessParking_garage              [False 'Unknown' nan 'None' True] 5
attr_BusinessParking_street              [False 'Unknown' nan 'None' True] 5
attr_BusinessParking_validated           [False 'Unknown' nan 'None' True] 5
attr_Busi

In [33]:
def fix_categorical_data(df, columns):
    for col in columns:
        unique_values = df[col].unique()
        u_rgx = re.compile("u\'([^\']+)\'")
        quote_rgx = re.compile("\'([^\']+)\'")
        
        for val in unique_values:
            if isinstance(val, bool):
                if str(val) == 'True':
                    df.loc[df[col].apply(lambda x: str(x)) == 'True', col] = 'True'
                elif str(val) == 'False':
                    df.loc[df[col].apply(lambda x: str(x)) == 'False', col] = 'False'
            
            if not isinstance(val, str):
                continue
                
            u_match = u_rgx.fullmatch(val)
            quote_match = quote_rgx.fullmatch(val)
            
            if u_match:
                df.loc[df[col] == val, col] = u_match.group(1)
            elif quote_match:
                df.loc[df[col] == val, col] = quote_match.group(1)

In [34]:
fix_categorical_data(business_norm, tCategorical_features)

Повеќето од атрибутите се номинални, па истата можеме да ја претвориме во нумеричка без да се грижиме за редоследот на вредностите. Како и да е, за да ја задржам конзистентноста за вредностите `Unknown` и `None` помеѓу колоните истите секаде ќе ги енкодирам со исти вредности.

In [35]:
ordinal_features = ['attr_RestaurantsPriceRange2', 'attr_NoiseLevel', 'attr_AgesAllowed']
nominal_features = [item for item in tCategorical_features if item not in ordinal_features ]
nominal_attrs = [item for item in normalized_attributes.columns.tolist() if item not in ordinal_features and item in business_norm.columns.tolist()]

In [36]:
def is_nan(val):
    try:
        return np.isnan(val)
    except:
        return False

def factorize_attributes(df, columns):
    for col in columns:
        values = {'Unknown': 8, 'None': 9}
        unique_values = df[col].unique()
        
        enumerated_remainings = lambda f,s: enumerate([item for item in f 
                                                       if item not in s and not is_nan(item)])
        
        if 'True' in unique_values or 'False' in unique_values:
            values.update({'True': 1, 'False': 0})
            values.update({val: 2 + i  for i, val in enumerated_remainings(unique_values, values)})
        else:
            values.update({val: i  for i, val in enumerated_remainings(unique_values, values)})
        
        df[col] = df[col].map(values)

In [37]:
factorize_attributes(business_norm, nominal_attrs)

In [38]:
business_norm.city = pd.factorize(business_norm.city)[0]
business_norm.state = pd.factorize(business_norm.state)[0]
business_norm.postal_code = pd.factorize(business_norm.postal_code)[0]

In [39]:
hour_values = set()
for col in normalized_hours:
    hour_values.update(normalized_hours[col].unique().tolist())
    
hour_values = sorted([item for item in hour_values if not is_nan(item)])
hours_map = {val: i+1 for i, val in enumerate(hour_values)}
hours_map['Closed'] = 0

In [40]:
for day in normalized_hours.columns.tolist():
    business_norm[day] = business_norm[day].map(hours_map)

Променливите `attr_RestaurantsPriceRange2`, `attr_NoiseLevel` и `attr_AgesAllowed` имаат одреден редослед, па истиот ќе се обидеме да го задржиме.

In [41]:
price_map = {'1': 0, '2': 1, '3': 2, '4': 3, 'Unknown': 8, 'None': 9}
noise_map = {'quiet': 0, 'average': 1, 'loud': 3, 'very_loud': 3, 'Unknown': 8, 'None': 9}
ages_map = {'18plus': 0, '19plus': 1, '21plus': 2, 'allages': 3, 'Unknown': 8, 'None': 9}
business_norm.attr_RestaurantsPriceRange2 = business_norm.attr_RestaurantsPriceRange2.map(price_map)
business_norm.attr_NoiseLevel = business_norm.attr_NoiseLevel.map(noise_map)
business_norm.attr_AgesAllowed = business_norm.attr_AgesAllowed.map(ages_map)

In [42]:
for col in normalized_categories.columns.tolist():
    business_norm[col] = business_norm[col].sparse.to_dense()

Отстранување на колони каде 95%+ од вредностите недостасуваат

In [43]:
def filter_empty_columns(df, empty_percent=.95):
    df = df.copy()
    column_status = df.isna().sum()
    column_count = (df.shape[1], df.shape[1])
    
    for col in df:
        if column_status[col] > (df.shape[0] * empty_percent):
            df = df.drop(col, axis=1)
            
    column_count = (column_count[0], df.shape[1])
    print('Columns:', column_count[0], '->', column_count[1])
    
    return df

In [44]:
business_norm = filter_empty_columns(business_norm)

Columns: 1435 -> 1435


За да одредиме кои колони да ги земеме при пополнувањето на вредностите коишто недостасуваат ќе користиме корелација. Имено, доколку корелираноста помеѓу променливите е поголема од 0.5 истата можеме да ја користиме за пополнување.  

In [45]:
categorical_features = [col for col in business_norm.columns.tolist() if col in normalized_attributes.columns.tolist() 
                        or col in normalized_categories.columns.tolist() 
                        or col in normalized_hours.columns.tolist()
                        or col in ['city', 'state', 'postal_code', 'is_open']]
numerical_features = ['latitude', 'longitude', 'stars', 'review_count']

In [117]:
def pairwise_apply(df, f, tril=False):
    def func(x, y):
        x_name, y_name = x.name, y.name
#         print(x_name, y_name)
        helper_df = pd.concat([x.rename('x'), y.rename('y')], axis=1)
        helper_df = helper_df.dropna()
        cols = helper_df.columns.tolist()
        
        return f(helper_df[cols[0]], helper_df[cols[1]])

    if not tril:
        return pd.DataFrame({j: {i: func(df[i], df[j]) for i in df} for j in df}) 
    
    pd_dict = {}
    
    for j, col_y in enumerate(df): # col
        for i, col_x in enumerate(df): # row
            if i >= j:
                pd_dict[col_y] = {} if col_y not in pd_dict else pd_dict[col_y]
                pd_dict[col_y][col_x] = func(df[col_x], df[col_y])
                
    return  pd.DataFrame(pd_dict)

In [118]:
business_cat = business_norm[categorical_features]

In [ ]:
# %%pixie_debugger
# cramers_v_association = pairwise_apply(business_cat.drop('postal_code', axis=1), f=cramers_v, tril=True)
theils_u_coef = pairwise_apply(business_cat.drop('postal_code', axis=1), f=theils_u, tril=False)

In [ ]:
non_predictor_col_names = ['business_id', 'name', 'address', 'latitude', 'longitude']
non_predictor_columns = business_norm[non_predictor_col_names]
business_norm = business_norm.drop(non_predictor_col_names, axis=1)

In [ ]:
predictors_list = business_norm.columns.tolist()

In [ ]:
cat_strategy = {col:'binary logistic' for col in normalized_categories.columns.tolist()}
attr_strategy = {col:'multinomial logistic' for col in normalized_attributes.columns.tolist()}
hours_strategy = {col:'multinomial logistic' for col in normalized_hours.columns.tolist()}
columns_strategy = {**cat_strategy, **attr_strategy, **hours_strategy}

In [ ]:
# data_sample = business_norm.sample(frac=0.1)

In [ ]:
# predictors_list = [col for col in business_norm.columns.tolist()]
# predictors_list = [col for col in predictors_list if col not in ['attr_BusinessAcceptsCreditCards', 'latitude', 'longitude']]

In [ ]:
# data_sample = data_sample[predictors_list + ['attr_BusinessAcceptsCreditCards']]

In [ ]:
import pixiedust # Debugger

In [ ]:
%%pixie_debugger
mi = MultipleImputer(n=1, strategy=columns_strategy, predictors='all', return_list=True)
mi_data_full = mi.fit_transform(business_norm)
# mi_data_full = mi.fit_transform(data_sample)
print("done")

In [ ]:
print('{:<40} {:>8} {:1}'.format('attr_BusinessAcceptsCreditCards', str(mi_data_full[0][1]['attr_BusinessAcceptsCreditCards'].unique()), len(mi_data_full[0][1]['attr_BusinessAcceptsCreditCards'].unique())))

<hr>

In [ ]:
%pixie_debugger

## Test autoimpute

In [ ]:
predictors_list = business_norm.columns.tolist()

In [ ]:
cat_strategy = {col:'bayesian binary logistic' for col in normalized_categories.columns.tolist()}
attr_strategy = {col:'multinomial logistic' for col in normalized_attributes.columns.tolist()}
hours_strategy = {col:'multinomial logistic' for col in normalized_hours.columns.tolist()}
columns_strategy = {**cat_strategy, **attr_strategy, **hours_strategy}

In [ ]:
data_sample = business_norm.sample(frac=0.1)

In [ ]:
predictors_list = [col for col in business_norm.columns.tolist() if col not in normalized_hours.columns.tolist() + normalized_attributes.columns.tolist()]
predictors_list = [col for col in predictors_list if col not in ['ctg_ATVRentalsTours', 'latitude', 'longitude']]

In [ ]:
data_sample = data_sample[predictors_list + ['ctg_ATVRentalsTours']]

In [ ]:
import pixiedust # Debugger

In [ ]:
for col in normalized_categories.columns.tolist():
    data_sample[col] = data_sample[col].sparse.to_dense()

In [ ]:
%%pixie_debugger
mi = MultipleImputer(n=2, strategy={'ctg_ATVRentalsTours': 'binary logistic'}, predictors=predictors_list, return_list=True)
# mi_data_full = mi.fit_transform(business_norm)
mi_data_full = mi.fit_transform(data_sample)
print("done")

In [ ]:
print('{:<40} {:>8} {:1}'.format('attr_BusinessAcceptsCreditCards', str(mi_data_full[0][1]['attr_BusinessAcceptsCreditCards'].unique()), len(mi_data_full[0][1]['attr_BusinessAcceptsCreditCards'].unique())))

In [ ]:
%pixie_debugger

In [ ]:
for cat in normalized_categories.columns.tolist():
    data_sample[cat] = data_sample[cat].astype(pd.SparseDtype(np.bool_, False))
#     data_sample[cat] = data_sample[cat].sparse.to_dense()

#### autoimpute test end

### Претворање на атрибутите во нумерички

Пред пополнување на вредностите кои недостасуваат, со цел полесно справување со истите, атрибутите ќе ги трансформираме во нумерички. Покрај полесно справување со вредностите коишто недостасуваат, претворањето на атрибутите во нумерички ќе ни овозможи полесно анализирање, но и примена на алгоритми кои работат само со ваквите атрибути. Како и да е, со цел задржување на што е можно повеќе информации потребно е да обрнеме внимание на тоа дали атрибутите се ординални или пак номинални. Вредности на ординалните атрибути ќе ги претворам во подредени броеви.

Подолу се претставени вредностите коишто ги имаат секоја од категориските колони чии вредности треба да се трансформираат.

In [ ]:
tCategorical_features = ['city', 'state'] + [col for col in normalized_attributes.columns.tolist() if col in business_norm.columns.tolist()]

In [ ]:
filled_attr = business_norm[tCategorical_features]
for col in business_norm.columns.tolist():
    cnt = len(business_norm[col].unique())
    if cnt <= 2:
        print('{:<40} {:>8} {:1}'.format(col, str(business_norm[col].unique()), cnt))

In [ ]:
def fix_categorical_data(df, columns):
    for col in columns:
        unique_values = df[col].unique()
        u_rgx = re.compile("u\'([^\']+)\'")
        quote_rgx = re.compile("\'([^\']+)\'")
        
        for val in unique_values:
            if isinstance(val, bool):
                if str(val) == 'True':
                    df.loc[df[col].apply(lambda x: str(x)) == 'True', col] = 'True'
                elif str(val) == 'False':
                    df.loc[df[col].apply(lambda x: str(x)) == 'False', col] = 'False'
            
            if not isinstance(val, str):
                continue
                
            u_match = u_rgx.fullmatch(val)
            quote_match = quote_rgx.fullmatch(val)
            
            if u_match:
                df.loc[df[col] == val, col] = u_match.group(1)
            elif quote_match:
                df.loc[df[col] == val, col] = quote_match.group(1)

In [ ]:
fix_categorical_data(business_norm, tCategorical_features)

Повеќето од атрибутите се номинални, па истата можеме да ја претвориме во нумеричка без да се грижиме за редоследот на вредностите. Како и да е, за да ја задржам конзистентноста за вредностите `Unknown` и `None` помеѓу колоните истите секаде ќе ги енкодирам со исти вредности.

In [ ]:
ordinal_features = ['attr_RestaurantsPriceRange2', 'attr_NoiseLevel', 'attr_AgesAllowed']
nominal_features = [item for item in tCategorical_features if item not in ordinal_features ]
nominal_attrs = [item for item in normalized_attributes.columns.tolist() if item not in ordinal_features and item in business_norm.columns.tolist()]

In [ ]:
def is_nan(val):
    try:
        return np.isnan(val)
    except:
        return False

def factorize_attributes(df, columns):
    for col in columns:
        values = {'Unknown': 8, 'None': 9}
        unique_values = df[col].unique()
        
        enumerated_remainings = lambda f,s: enumerate([item for item in f 
                                                       if item not in s and not is_nan(item)])
        
        if 'True' in unique_values or 'False' in unique_values:
            values.update({'True': 1, 'False': 0})
            values.update({val: 2 + i  for i, val in enumerated_remainings(unique_values, values)})
        else:
            values.update({val: i  for i, val in enumerated_remainings(unique_values, values)})
        
        df[col] = df[col].map(values)

In [ ]:
factorize_attributes(business_norm, nominal_attrs)

In [ ]:
business_norm.city = pd.factorize(business_norm.city)[0]
business_norm.state = pd.factorize(business_norm.state)[0]
business_norm.postal_code = pd.factorize(business_norm.postal_code)[0]

In [ ]:
hour_values = set()
for col in normalized_hours:
    hour_values.update(normalized_hours[col].unique().tolist())
    
hour_values = sorted([item for item in hour_values if not is_nan(item)])
hours_map = {val: i+1 for i, val in enumerate(hour_values)}
hours_map['Closed'] = 0

In [ ]:
for day in normalized_hours.columns.tolist():
    business_norm[day] = business_norm[day].map(hours_map)

Променливите `attr_RestaurantsPriceRange2`, `attr_NoiseLevel` и `attr_AgesAllowed` имаат одреден редослед, па истиот ќе се обидеме да го задржиме.

In [ ]:
price_map = {'1': 0, '2': 1, '3': 2, '4': 3, 'Unknown': 8, 'None': 9}
noise_map = {'quiet': 0, 'average': 1, 'loud': 3, 'very_loud': 3, 'Unknown': 8, 'None': 9}
ages_map = {'18plus': 0, '19plus': 1, '21plus': 2, 'allages': 3, 'Unknown': 8, 'None': 9}
business_norm.attr_RestaurantsPriceRange2 = business_norm.attr_RestaurantsPriceRange2.map(price_map)
business_norm.attr_NoiseLevel = business_norm.attr_NoiseLevel.map(noise_map)
business_norm.attr_AgesAllowed = business_norm.attr_AgesAllowed.map(ages_map)

In [ ]:
[]

In [ ]:
col_types = {col: 'Int64' for col in business_norm.columns.tolist()[11:]}

In [ ]:
for key, value in col_types.items():
    try:
        business_norm[key] = business_norm[key].astype('Int64')
    except Exception as e:
        business_norm[key] = business_norm[key].astype('Sparse[int]')
        print(key)

In [ ]:
business_norm.dtypes

In [ ]:
business_norm['ctg_UniversityHousing'].dtype